In [6]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('data/two_treatment/dt0_without_context.csv' ,index_col=0)
df.head()

,t,patient_id,treatment,outcome
0,0,0,0,0.125730
1,1,0,0,-0.132105
2,2,0,0,0.640423
3,3,0,0,0.104900
4,4,0,0,-0.535669


#### creating random nan values

In [81]:
column = 'outcome'
nan_fraction = 0.1  # 30% of the data in column B will be set to NaN

total_rows = len(df)
nan_count = int(total_rows * nan_fraction)

# Randomly choose row indices where NaN should be inserted in the column
nan_indices = np.random.choice(df.index, nan_count, replace=False)

# Set those randomly selected positions to NaN in the specified column
df.loc[nan_indices, column] = np.nan


#### Separating the dataframe to calculate missing data
In a bayesian aaptive N_of_1 trials, we are going to calculate the missing data as we prepare to update the posterior joint distribution. Thus, the first step is to sepaate the dataframe when the first missing value appears.

In [82]:
## seperating the dataframe at the point of first missing value
nan_t = df[df.isna().any(axis=1)]['t']
nan_t = nan_t.sort_values(ascending=True).unique() ## shorting the value in ascending order to make sure we dot the first time cycle
ts = nan_t[0]
dt = df[df['t'] <= ts].copy()
dt.head()
     

,t,patient_id,treatment,outcome
0,0,0,0,0.125730
30,0,1,0,0.345584
60,0,2,0,0.189053
90,0,3,0,2.040919
120,0,4,0,NaN


Now, we have data set where we encountered the first missig value in the experiments. lets fill the missing value with various method.
#### KNN mean filling method
For the context filling method, we are followin a similar algorithm to KNN imputation method. We can descide how many value we want to consider. if 1, then the nearest distance value will fill the missng value. If more than 1 (i.e K), then the mean value of K nearest values will fill the missing value.
The context of the each paitent and the treatment altogether is a vector (i.e for paitient id 1 the vector is $[c_1, c_2, ...., c_n, treatment]_{p_1}$). We calculate the distance of the missing value points/paitients vector with all the other available vector. Then, K nearest outcome values mean is used to fill the missing value.

If no context is available, then we are using the treatment as a singular context.

In [83]:
def get_context_vectors(without_miss_dt, context_cols):
        
    context_vectors = []
    for row in without_miss_dt.itertuples():
        row_values = [getattr(row, col) for col in context_cols]
        context_vectors.append(row_values)

    return (np.array(context_vectors))

def get_distance(context_vectors, miss_vec):
    
    distances = []
    for vector in context_vectors:
        d = np.linalg.norm(vector - miss_vec)
        distances.append(d) 
    return(distances)

In [84]:
def KNN_fill(dt, context_cols = list, k = 1): 
    # dt is the data Frame
    # context_cols will be list of columns name which are considered as context 
    
    dt_fill = dt.copy()
    miss_dt = dt_fill[dt_fill['outcome'].isna()]
    without_miss_dt = dt_fill[dt_fill['outcome'].isna() == False].copy()
    context_vectors = get_context_vectors(without_miss_dt, context_cols)
    
    for row in miss_dt.itertuples():
        miss_vec = np.array([getattr(row, col) for col in context_cols])
        dis = get_distance(context_vectors, miss_vec)
        without_miss_dt['distance'] = dis
        
        sorted_without_miss_dt = without_miss_dt.sort_values(by = 'distance')
        
        fill_value = sorted_without_miss_dt['outcome'].head(k).mean()
        dt_fill.loc[row.Index,'outcome'] = fill_value
    
    return dt_fill

In [85]:
k = 2
context_cols = ['treatment']

print(dt.isna().value_counts())
filled_dt = KNN_fill(dt, context_cols, k)
print(filled_dt.isna().value_counts())


t      patient_id  treatment  outcome
False  False       False      False      91
                              True        9
Name: count, dtype: int64
t      patient_id  treatment  outcome
False  False       False      False      100
Name: count, dtype: int64


**when t =! 0**

In [86]:
next_cycle = df[(df['t']> nan_t[0]) & (df['t'] <= nan_t[1])].copy()
next_dt = pd.concat([filled_dt, next_cycle], axis= 0, ignore_index=False) # add the next cycle to the data
next_dt = next_dt.sort_index()
next_dt

,t,patient_id,treatment,outcome
0,0,0,0,0.125730
1,1,0,0,-0.132105
30,0,1,0,0.345584
31,1,1,0,0.821618
60,0,2,0,0.189053
...,...,...,...,...
2911,1,97,1,1.160014
2940,0,98,0,-1.087591
2941,1,98,0,-1.244621
2970,0,99,0,0.082494


In [87]:
print(next_dt.isna().value_counts())
next_filled_dt = KNN_fill(next_dt, context_cols, k)
print(next_filled_dt.isna().value_counts())

t      patient_id  treatment  outcome
False  False       False      False      193
                              True         7
Name: count, dtype: int64
t      patient_id  treatment  outcome
False  False       False      False      200
Name: count, dtype: int64
